In [4]:
!mkdir -p 'db/17/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'db/17/paul_graham_essay.txt'

--2023-12-12 01:52:30--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘db/17/paul_graham_essay.txt’

db/17/paul_graham_e 100%[===================>]  73.28K  --.-KB/s    in 0.06s   

2023-12-12 01:52:30 (1.17 MB/s) - ‘db/17/paul_graham_essay.txt’ saved [75042/75042]



In [13]:
import re, yaml, os
from time import time
from FlagEmbedding import FlagReranker
from llama_index.llms import AzureOpenAI
from llama_index.llm_predictor import LLMPredictor
from llama_index import set_global_service_context
from llama_index.embeddings import FastEmbedEmbedding
from llama_index.postprocessor import SentenceTransformerRerank
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

In [4]:
with open('/Users/1zuu/Desktop/LLM RESEARCH/RAG UNIVERSITY/cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['AD_OPENAI_API_KEY'] = credentials['AD_OPENAI_API_KEY']
os.environ['HUGGINGFACEHUB_API_TOKEN'] = credentials['HUGGINGFACEHUB_API_TOKEN']

In [5]:
embedding_llm = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
llm=AzureOpenAI(
                deployment_name=credentials['AD_DEPLOYMENT_ID'],
                model=credentials['AD_ENGINE'],
                api_key=credentials['AD_OPENAI_API_KEY'],
                api_version=credentials['AD_OPENAI_API_VERSION'],
                azure_endpoint=credentials['AD_OPENAI_API_BASE']
                )
chat_llm = LLMPredictor(llm)

service_context = ServiceContext.from_defaults(
                                                embed_model=embedding_llm,
                                                llm_predictor=chat_llm
                                                )
set_global_service_context(service_context)

In [6]:
documents = SimpleDirectoryReader("./db/17").load_data()
len(documents)

1

In [7]:
index = VectorStoreIndex.from_documents(documents=documents)
index

In [18]:
rerank = SentenceTransformerRerank(
                                model="cross-encoder/ms-marco-MiniLM-L-2-v2", 
                                top_n=4
                                )

## With Reranker

In [19]:
query_engine = index.as_query_engine(
                                    similarity_top_k=10, 
                                    node_postprocessors=[rerank]
                                    )
now = time()
response = query_engine.query("Which grad schools did the author apply for and why?")
print(f"Elapsed: {round(time() - now, 2)}s")

Elapsed: 3.02s


In [20]:
print(response)

There is no information in the given context about which grad schools the author applied for and why.


In [21]:
print(response.get_formatted_sources(length=200))

> Source (Doc id: 528a77be-f37d-4cf2-86df-5c7a24701d33): I didn't want to drop out of grad school, but how else was I going to get out? I remember when my friend Robert Morris got kicked out of Cornell for writing the internet worm of 1988, I was envious...

> Source (Doc id: d7af7df1-f520-46c2-b35b-74a5c8dfbbd9): So I looked around to see what I could salvage from the wreckage of my plans, and there was Lisp. I knew from experience that Lisp was interesting for its own sake and not just for its association ...

> Source (Doc id: 997dee41-247f-4a85-85ff-7abbf4e14c09): They were an impressive group. That first batch included reddit, Justin Kan and Emmett Shear, who went on to found Twitch, Aaron Swartz, who had already helped write the RSS spec and would a few ye...

> Source (Doc id: 070afb33-7a4e-47a5-a5a1-3abf96e03aa3): What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what 

# Without Reranker

In [22]:
query_engine = index.as_query_engine(similarity_top_k=10)

now = time()
response = query_engine.query(
    "Which grad schools did the author apply for and why?",
)

print(f"Elapsed: {round(time() - now, 2)}s")

Elapsed: 4.48s


In [23]:
print(response.get_formatted_sources(length=200))

> Source (Doc id: 528a77be-f37d-4cf2-86df-5c7a24701d33): I didn't want to drop out of grad school, but how else was I going to get out? I remember when my friend Robert Morris got kicked out of Cornell for writing the internet worm of 1988, I was envious...

> Source (Doc id: 34a6172b-b687-40be-b88c-721b61e79406): So while working on things that aren't prestigious doesn't guarantee you're on the right track, it at least guarantees you're not on the most common type of wrong one.

Over the next several years ...

> Source (Doc id: b29a6395-3421-48c6-8312-e9f626914860): I couldn't have put this into words when I was 18. All I knew at the time was that I kept taking philosophy courses and they kept being boring. So I decided to switch to AI.

AI was in the air in t...

> Source (Doc id: 070afb33-7a4e-47a5-a5a1-3abf96e03aa3): What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what 